# Coursework 1: ML basics and fully-connected networks

#### Instructions

Please submit a version of this notebook containing your answers on CATe as *CW1*. Write your answers in the cells below each question.

We recommend that you work on the Ubuntu workstations in the lab. This assignment and all code were only tested to work on these machines. In particular, we cannot guarantee compatibility with Windows machines and cannot promise support if you choose to work on a Windows machine.

You can work from home and use the lab workstations via ssh (for list of machines: https://www.doc.ic.ac.uk/csg/facilities/lab/workstations). 

Once logged in, run the following commands in the terminal to set up a Python environment with all the packages you will need.

    export PYTHONUSERBASE=/vol/bitbucket/nuric/pypi
    export PATH=/vol/bitbucket/nuric/pypi/bin:$PATH

Add the above lines to your `.bashrc` to have these enviroment variables set automatically each time you open your bash terminal.

Any code that you submit will be expected to run in this environment. Marks will be deducted for code that fails to run.

Run `jupyter-notebook` in the coursework directory to launch Jupyter notebook in your default browser.

DO NOT attempt to create a virtualenv in your home folder as you will likely exceed your file quota.

**DEADLINE: 7pm, Tuesday 5th February, 2019**

## Part 1

1. Describe two practical methods used to estimate a supervised learning model's performance on unseen data. Which strategy is most commonly used in most deep learning applications, and why?
2. Suppose that you have reason to believe that your multi-layer fully-connected neural network is overfitting. List four things that you could try to improve generalization performance.

## Part 2

1. Why can gradient-based learning be difficult when using the sigmoid or hyperbolic tangent functions as hidden unit activation functions in deep, fully-connected neural networks?
2. Why is the issue that arises in the previous question less of an issue when using such functions as output unit activation functions, provided that an appropriate loss function is used?
3. What would happen if you initialize all the weights to zero in a multi-layer fully-connected neural network and attempt to train your model using gradient descent? What would happen if you did the same thing for a logistic regression model?

\**ANSWERS FOR PART 2 IN THIS CELL*\*

## Part 3

In this part, you will use PyTorch to implement and train a multinomial logistic regression model to classify MNIST digits.

Restrictions:
* You must use (but not modify) the code provided in `utils.py`. **This file is deliberately not documented**; read it carefully as you will need to understand what it does to complete the tasks.
* You are NOT allowed to use the `torch.nn` module.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultinomialLogisticRegressionClassifier` class below by filling in the missing parts (expected behaviour is prescribed in the documentation):
    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 92% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.


In [1]:
from utils import *

In [2]:
# *CODE FOR PART 3.1 IN THIS CELL*

class MultinomialLogisticRegressionClassifier:
    def __init__(self, weight_init_sd=100.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.weight_init_sd = weight_init_sd
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        self.size = 28*28
        self.class_size = 10
        weights = torch.normal(mean = 0, std = torch.zeros(self.size,self.class_size)+weight_init_sd)
        self.weights = weights.requires_grad_(True)
        bias = torch.normal(mean = 0, std = torch.zeros(self.class_size)+weight_init_sd)
        self.bias = bias.requires_grad_(True)
#         print("size of weights",self.weights.shape)
#         print("size of bias",self.bias.shape)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
    
    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be a Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        flattened_inputs = inputs.view(-1, 784)
        linear = flattened_inputs@self.weights+self.bias
        outputs =  F.log_softmax(linear,dim=1)
        return outputs
        
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
#         return self.weights, self.basis
        yield self.weights
        yield self.bias
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        l1_norm = 0
        for paras in self.parameters():
            l1_norm +=torch.sum(torch.abs(paras))
        return l1_norm
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 norm of the model's weight vector (i.e. 
        square root of sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        sum = 0
        for paras in self.parameters():
            sum +=torch.sum(paras**2)
        l2_norm = torch.sqrt(sum)
        return l2_norm
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [53]:
# *CODE FOR PART 3.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
model = MultinomialLogisticRegressionClassifier(weight_init_sd=0.01)
res = run_experiment(
    model,
    optimizer=optim.Adam(model.parameters(), lr = 0.001),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=0.0000001,
    l2_penalty_coef=0.0000001,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 0.5598, Accuracy: 0.8691
Validation set:	Average loss: 0.3591, Accuracy: 0.9003

Epoch 1: training...
Train set:	Average loss: 0.3301, Accuracy: 0.9096
Validation set:	Average loss: 0.3127, Accuracy: 0.9110

Epoch 2: training...
Train set:	Average loss: 0.3005, Accuracy: 0.9169
Validation set:	Average loss: 0.2909, Accuracy: 0.9185

Epoch 3: training...
Train set:	Average loss: 0.2858, Accuracy: 0.9206
Validation set:	Average loss: 0.2819, Accuracy: 0.9188

Epoch 4: training...
Train set:	Average loss: 0.2770, Accuracy: 0.9233
Validation set:	Average loss: 0.2748, Accuracy: 0.9218

Epoch 5: training...
Train set:	Average loss: 0.2703, Accuracy: 0.9245
Validation set:	Average loss: 0.2711, Accuracy: 0.9233

Epoch 6: training...
Train set:	Average loss: 0.2662, Accuracy: 0.9256
Validation set:	Average loss: 0.2685, Accuracy: 0.9235

Epoch 7: training...
Train set:	Average loss: 0.2622, Accuracy: 0.9275
Validation set:	Average loss: 0.2672, Ac

## Part 4

In this part, you will use PyTorch to implement and train a multi-layer fully-connected neural network to classify MNIST digits.

Your network must have three hidden layers with 128, 64, and 32 hidden units respectively.

The same restrictions as in Part 3 apply.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultilayerClassifier` class below by filling in the missing parts of the following methods (expected behaviour is prescribed in the documentation):

    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 97% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.

3. Describe an alternative strategy for initializing weights that may perform better than the strategy we have used here.

In [63]:
# *CODE FOR PART 4.1 IN THIS CELL*

class MultilayerClassifier:
    def __init__(self, activation_fun="sigmoid", weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        super().__init__()
        self.activation_fun = activation_fun
        self.weight_init_sd = weight_init_sd

        if self.activation_fun == "relu":
            self.activation = F.relu
        elif self.activation_fun == "sigmoid":
            self.activation = torch.sigmoid
        elif self.activation_fun == "tanh":
            self.activation = torch.tanh
        else:
            raise NotImplementedError()

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        cov1_weights = torch.normal(mean = 0, std = torch.zeros(784,128)+weight_init_sd)
        cov2_weights = torch.normal(mean = 0, std = torch.zeros(128,64)+weight_init_sd)
        cov3_weights = torch.normal(mean = 0, std = torch.zeros(64,32)+weight_init_sd)
        out_weights = torch.normal(mean = 0, std = torch.zeros(32,10)+weight_init_sd)
        
        cov1_bias = torch.normal(mean = 0, std = torch.zeros(128)+weight_init_sd)
        cov2_bias = torch.normal(mean = 0, std = torch.zeros(64)+weight_init_sd)
        cov3_bias = torch.normal(mean = 0, std = torch.zeros(32)+weight_init_sd)
        out_bias = torch.normal(mean = 0, std = torch.zeros(10)+weight_init_sd)
        
        self.cov1_weights = cov1_weights.requires_grad_(True)
        self.cov2_weights = cov2_weights.requires_grad_(True)
        self.cov3_weights = cov3_weights.requires_grad_(True)
        self.out_weights = out_weights.requires_grad_(True)
        self.cov1_bias = cov1_bias.requires_grad_(True)
        self.cov2_bias = cov2_bias.requires_grad_(True)
        self.cov3_bias = cov3_bias.requires_grad_(True)
        self.out_bias = out_bias.requires_grad_(True)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        flattened_inputs = inputs.view(-1, 784)
        cov1_result = self.activation(flattened_inputs @ self.cov1_weights + self.cov1_bias)
        cov2_result = self.activation(cov1_result @ self.cov2_weights + self.cov2_bias)
        cov3_result = self.activation(cov2_result @ self.cov3_weights + self.cov3_bias)
        outputs =  F.log_softmax((cov3_result @ self.out_weights + self.out_bias),dim=1)
        return outputs
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        yield self.cov1_weights
        yield self.cov1_bias
        yield self.cov2_weights
        yield self.cov2_bias
        yield self.cov3_weights
        yield self.cov3_bias
        yield self.out_weights
        yield self.out_bias
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        l1_norm = 0
        for paras in self.parameters():
            l1_norm +=torch.sum(torch.abs(paras))
        return l1_norm
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 norm of the model's weight vector (i.e. 
        square root of sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        sum = 0
        for paras in self.parameters():
            sum +=torch.sum(paras**2)
        l2_norm = torch.sqrt(sum)
        return l2_norm
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [68]:
# *CODE FOR PART 4.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *

model = MultilayerClassifier(activation_fun='relu', weight_init_sd=0.01)
res = run_experiment(
    model,
    optimizer=optim.Adam(model.parameters(), 0.001),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=0.000001,
    l2_penalty_coef=0.000001,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 0.8608, Accuracy: 0.7055
Validation set:	Average loss: 0.4235, Accuracy: 0.8780

Epoch 1: training...
Train set:	Average loss: 0.3120, Accuracy: 0.9098
Validation set:	Average loss: 0.2254, Accuracy: 0.9315

Epoch 2: training...
Train set:	Average loss: 0.1913, Accuracy: 0.9447
Validation set:	Average loss: 0.1653, Accuracy: 0.9515

Epoch 3: training...
Train set:	Average loss: 0.1420, Accuracy: 0.9592
Validation set:	Average loss: 0.1401, Accuracy: 0.9553

Epoch 4: training...
Train set:	Average loss: 0.1112, Accuracy: 0.9669
Validation set:	Average loss: 0.1364, Accuracy: 0.9583

Epoch 5: training...
Train set:	Average loss: 0.0900, Accuracy: 0.9737
Validation set:	Average loss: 0.1210, Accuracy: 0.9640

Epoch 6: training...
Train set:	Average loss: 0.0732, Accuracy: 0.9779
Validation set:	Average loss: 0.1200, Accuracy: 0.9647

Epoch 7: training...
Train set:	Average loss: 0.0621, Accuracy: 0.9816
Validation set:	Average loss: 0.1068, Ac

\**ANSWERS FOR PART 4.3 IN THIS CELL*\*